In [1]:
from pydap.client import open_url
from datetime import datetime
import numpy as np
import pandas as pd
import time
from datetime import timedelta
import pyproj
import xarray as xr
from scipy.interpolate import LinearNDInterpolator
import matplotlib.pyplot as plt
import math
import pickle
from datetime import date
import os
import plotly.express as px
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib
import cartopy as cart

def retrieve_microplastics_gt():
    lats = np.zeros(181)
    count = 90
    for i in range(len(lats)):
        lats[i] = count
        count -= 1
    for filename in os.listdir('microplast_gt'):
        logfilename = filename[:-4] + '_log'
        try:
            df = pd.read_csv('microplast_gt/' + filename, header=None)
        except:
            print(filename)
            continue
        longlist = []
        latlist = []
        valuelist_log = []
        valuelist = []
        for long in range(0, 361):
            for lat in range(0, 181):
                val = df[long][lat]
                if val < 1:
                    val = np.nan
                longlist.append(long)
                latlist.append(lats[lat])
                valuelist_log.append(np.log10(val))
                valuelist.append(val)
        if filename == "lebretonmodel_abundance.csv":
            res_df = pd.DataFrame({'sp_lon' : longlist, 'sp_lat' : latlist, filename[:-4] : valuelist, logfilename : valuelist_log})
        else:
            res_df[filename[:-4]] = valuelist
            res_df[logfilename] = valuelist_log
    return res_df

def plot_wind(df, var1, var2 ,interpolated = False):

    # Settings for the plot
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    lon_formatter = LongitudeFormatter()
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.gridlines(draw_labels=True, alpha=0.5)
    plt.scatter(df['sp_lon'], df['sp_lat'], c=list(np.sqrt(df[var1]**2 + df[var2]**2)), cmap='RdBu')
    bar = plt.colorbar(pad=0.15, orientation='horizontal')
    bar.ax.set_title('m/s')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    if interpolated:
        plt.savefig('wind_speed_inter.png')
    else:
        plt.savefig('wind_speed.png')
    plt.show()
    
def plot_var(df, var):

    # Settings for the plot
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cart.feature.LAND, zorder=100, edgecolor='k')
    ax.gridlines(draw_labels=True, alpha=0.5)
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    plt.scatter(df['sp_lon'], df['sp_lat'], c=list(df[var]))
    bar = plt.colorbar(pad=0.15, orientation='horizontal')
    plt.title(var)
    bar.ax.set_title('Power 10^')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.savefig('nbrcs-diff.svg')
    plt.show()
    
def generate_url(year, month, day, satellite_number):

    day_of_year = datetime(year, month, day).timetuple().tm_yday
    date_string = str(year) + str(month).zfill(2) + str(day).zfill(2)

    base_url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/cygnss/L1/v3.0/'
    specific_url = str(year) + '/' + str(day_of_year).zfill(3) + '/cyg0' + str(satellite_number) + '.ddmi.s' + \
                   date_string + '-000000-e' + date_string + '-235959.l1.power-brcs.a30.d31.nc'
    data_url = base_url + specific_url
    clickable_url = base_url + specific_url + '.html'

    return data_url + '?sp_lat,sp_lon,track_id,quality_flags,ddm_timestamp_utc,ddm_nbrcs,fresnel_coeff,sp_inc_angle', clickable_url

def fetch_cygnss(y1, m1, d1, y2, m2, d2):
    sdate = date(y1, m1, d1)   # start date
    edate = date(y2, m2, d2)   # end date
    delta = edate - sdate       # as timedelta
    df_list = []
    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        df = get_cygnss_data(day.year, day.month, day.day)
        df = prep_cygnss(df)
        df_list.append(df)
        df.to_csv("cygnss_data_whole_world/" + str(day.year) + str(day.month) + str(day.day) + ".csv" ,index=False)
    if len(df_list) == 1:
        return df_list[0]
    else:
        return pd.concat(df_list)
    
def hours_since_ref(year, month, day):
    d0 = date(1992, 10, 5)
    d1 = date(year, month, day)
    delta = d1 - d0
    hours = delta.days*24
    return hours

def get_cygnss_data(year, month, day):
    cygnss_df = pd.DataFrame()
    hours = hours_since_ref(year, month, day)
    for sat_numb in range(1, 9):  # Remember to change back to  1, 9
        print("Satellite number : " + str(sat_numb))
        test_data_url, test_clickable_url = generate_url(year, month, day, sat_numb)
        dataset = open_url(test_data_url, output_grid=False)
        for ddm in range(4):  # Remember to change back to 4
            ddm_df = pd.DataFrame()
            print("ddm : " + str(ddm))

            ddm_timestamp_utc = np.array(dataset.ddm_timestamp_utc[:, ddm])
            ddm_timestamp_utc = np.rint(ddm_timestamp_utc / 3600) + hours

            ddm_df['sp_lat'] = np.array(dataset.sp_lat[:, ddm]).tolist()
            sp_lon = np.array(dataset.sp_lon[:, ddm]).tolist()
            ddm_df['sp_lon'] = sp_lon
            ddm_df['sp_lon'] = np.array(dataset.sp_lon[:, ddm]).tolist()
            ddm_df['hours_since_ref'] = ddm_timestamp_utc.tolist()
            ddm_df['ddm_nbrcs'] = np.array(dataset.ddm_nbrcs[:, ddm]).tolist()
            ddm_df['fresnel_coeff'] = np.array(dataset.fresnel_coeff[:, ddm]).tolist()
            ddm_df['sp_inc_angle'] = np.array(dataset.sp_inc_angle[:, ddm]).tolist()
            ddm_df['quality_flags'] = np.array(dataset.quality_flags[:, ddm]).tolist()
            ddm_df['track_id'] = np.array(dataset.track_id[:, ddm]).tolist()
            
            for col in ddm_df.columns:
                if col != 'ddm_channel' and col != 'hours_since_ref' and col != 'unique_track_id':
                    ddm_df[col] = ddm_df[col].apply(lambda x: x[0])
            hoursince = [str(hours)] * len(sp_lon)
            sat_numb_ar = [str(sat_numb)] * len(sp_lon)
            ddm_df['unique_track_id'] = generate_unique_track_id_value(hoursince, list(map(str, ddm_df['track_id'])), 
                                                                       sat_numb_ar)
            cygnss_df = cygnss_df.append(ddm_df, ignore_index=True)
    return cygnss_df


def generate_unique_track_id_value(hoursince, track_id, sat_nr):
    return list(map(''.join, zip(*[hoursince, track_id, sat_nr])))

def open_oskar_data_local(filename):
    ds = xr.open_dataset(filename)
    oskar_df = ds.to_dataframe()
    oskar_df.dropna(inplace = True)
    oskar_df = oskar_df.reset_index()
    d0 = date(1992, 10, 5)
    d1 = oskar_df['time'][1].date()
    delta = d1 - d0
    hours = delta.days*24
    oskar_df['time'] = np.zeros(len(oskar_df['time']))
    oskar_df = oskar_df.assign(time=hours)
    oskar_df = oskar_df.rename(columns={"latitude": "sp_lat", "longitude": "sp_lon", "time" : "hours_since_ref"})
    oskar_df = oskar_df.loc[oskar_df['sp_lon'] < 380 ]
    oskar_df['sp_lon'] = oskar_df['sp_lon']%360
    oskar_df = reduce_area_of_df(oskar_df)
    return oskar_df


def fetch_all_oskar_files():
    df_list = []
    for filename in os.listdir('oskar_data'):
        df_list.append(open_oskar_data_local('oskar_data/' + filename))
    if len(df_list) == 1:
        return df_list[0]
    else:
        return pd.concat(df_list)

def open_cygnss_csvs():
    df_list = []
    for filename in os.listdir('cygnss_data'):
        df_list.append(pd.read_csv('cygnss_data/' + filename))
    if len(df_list) == 1:
        return df_list[0]
    else:
        return pd.concat(df_list)
    
def open_anomalies_csvs():
    df_list = []
    for filename in os.listdir('mss_ano_df'):
        df_list.append(pd.read_csv('mss_ano_df/' + filename))
    if len(df_list) == 1:
        return df_list[0]
    else:
        return pd.concat(df_list)

def prep_cygnss(cygnss_df):
    np.warnings.filterwarnings('ignore')
    for key in cygnss_df:
        cygnss_df = cygnss_df[cygnss_df[key] != -9999.0]
    cygnss_df = cygnss_df[cygnss_df.quality_flags%2 == 0]
    cygnss_df.dropna(inplace = True)
    cygnss_df['nbrcs_log'] = 10*np.log10(cygnss_df.ddm_nbrcs.to_numpy())
    cygnss_df.drop('ddm_nbrcs', inplace=True, axis=1)
    return cygnss_df

# SET AREA, Function Is called when extracting CYGNSS, OSKAR and ERA5
def reduce_area_of_df(df):
    df = df[df.sp_lat <= 40]
    df = df[df.sp_lat >= -40]
    
    df_northern = df[df.sp_lat >= 20]
    df_southern = df[df.sp_lat <= -20]
    df = pd.concat([df_northern,df_southern])
    df = df[df.sp_lon >= 220]
    return df[df.sp_lon <= 260]

def get_era_5(filename):
    ds = xr.open_dataset(filename)
    era_5_df = ds.to_dataframe()
    index_long = era_5_df.index.levels[0]
    index_lat = era_5_df.index.levels[1]
    index_time = era_5_df.index.levels[2]

    start_time = pd.Timestamp(1992, 10, 5)
    index_time_fixed = []
    for tid in index_time:
        hours = tid - start_time
        index_time_fixed.append(hours.days*24 + hours.seconds/3600)

    long, lat, time = np.meshgrid(index_long, index_lat, index_time_fixed, indexing = 'ij')

    long = long.flatten()
    lat = lat.flatten()
    time = time.flatten()

    return pd.DataFrame({'sp_lon': long+360, 'sp_lat' :lat, 'hours_since_ref': time, 'u10': era_5_df["u10"].to_numpy()
                         ,'v10': era_5_df["v10"].to_numpy()})

In [ ]:
oskar_df = fetch_all_oskar_files()
oskar_df

In [ ]:
read = False
if read:
    cygnss_df = open_cygnss_csvs()
    # If you want to change area
    cygnss_df = reduce_area_of_df(cygnss_df)
else:
    cygnss_df = fetch_cygnss(2021, 11, 7, 2021, 11, 8)
    cygnss_df = reduce_area_of_df(cygnss_df)
cygnss_df

In [ ]:
cygnss_df.unique_track_id

In [ ]:
era_5_df = get_era_5('era_5_wind/wind_one_week.nc')
era_5_df

In [21]:
ds = xr.open_dataset('era_5_wind/wind_one_week.nc')
era_5_df = ds.to_dataframe()
era_5_df

def reduce_area_of_df_era_5(df):
    df = df[df.index.get_level_values(0) + 360 >= 220]
    df = df[df.index.get_level_values(0) + 360 <= 260]
    df = df[df.index.get_level_values(1) <= 40]
    df = df[df.index.get_level_values(1) >= -40]

    df_northern = df[df.index.get_level_values(1) >= 20]
    df_southern = df[df.index.get_level_values(1) <= -20]
    return pd.concat([df_northern, df_southern])
reduce_area_of_df_era_5(era_5_df)

,,,u10,v10
longitude,latitude,time,,


In [ ]:
interp_u10 = LinearNDInterpolator(list(zip(era_5_df['sp_lon'], era_5_df['sp_lat'], era_5_df['hours_since_ref'])), era_5_df['u10'])
print("done")
interp_v10 = LinearNDInterpolator(list(zip(era_5_df['sp_lon'], era_5_df['sp_lat'], era_5_df['hours_since_ref'])), era_5_df['v10'])

In [ ]:
lons_to_interpolate = cygnss_df["sp_lon"].to_numpy()
lats_to_interpolate = cygnss_df["sp_lat"].to_numpy()
times_to_interpolate = cygnss_df["hours_since_ref"].to_numpy()

u10 = interp_u10(lons_to_interpolate, lats_to_interpolate, times_to_interpolate)
v10 = interp_v10(lons_to_interpolate, lats_to_interpolate, times_to_interpolate)

In [ ]:
cygnss_and_interp_df = pd.DataFrame({'sp_lon': lons_to_interpolate, 'sp_lat' :lats_to_interpolate, 'hours_since_ref': times_to_interpolate, 'u10': u10, 'v10': v10, 'nbrcs_log' : cygnss_df['nbrcs_log'], 'fresnel_coeff' : cygnss_df['fresnel_coeff'], 'sp_inc_angle' : cygnss_df['sp_inc_angle']  })
cygnss_and_interp_df

In [ ]:
interp_u = LinearNDInterpolator(list(zip(oskar_df['sp_lat'], oskar_df['sp_lon'], oskar_df['hours_since_ref'])), oskar_df['u'])
print("done")
interp_v = LinearNDInterpolator(list(zip(oskar_df['sp_lat'], oskar_df['sp_lon'], oskar_df['hours_since_ref'])), oskar_df['v'])

In [ ]:
lons_to_interpolate = cygnss_df["sp_lon"].to_numpy()
lats_to_interpolate = cygnss_df["sp_lat"].to_numpy()
times_to_interpolate = cygnss_df["hours_since_ref"].to_numpy()

u_current = interp_u(lats_to_interpolate, lons_to_interpolate, times_to_interpolate)
v_current = interp_v(lats_to_interpolate, lons_to_interpolate, times_to_interpolate)

cygnss_and_interp_df['u'] = u_current
cygnss_and_interp_df['v'] = v_current
cygnss_and_interp_df

In [ ]:
diff_u = cygnss_and_interp_df['u10']-cygnss_and_interp_df['u']
diff_v = cygnss_and_interp_df['v10']-cygnss_and_interp_df['v']
delta = np.sqrt(diff_u**2 + diff_v**2)

top_frac = 0.059*delta**3 - 0.147*delta**2 + 0.903**delta + 1.389
bot_frac = delta**3 + 18.161*delta**2 - 117.602*delta + 706.9
mss_from_wind_current = top_frac/bot_frac
fresnell_sqrd = cygnss_and_interp_df['fresnel_coeff'].to_numpy()**2

bias_df = pd.read_csv ('bias_model.csv')
interp_bias = LinearNDInterpolator(list(zip(bias_df['inc_angle'], bias_df['delta'])), bias_df['bias'])
biases = interp_bias(cygnss_and_interp_df['sp_inc_angle'], delta)
predicted_nbrcs = 10*np.log10(fresnell_sqrd/mss_from_wind_current) + biases

mss_from_cygnss = fresnell_sqrd/(10**((cygnss_and_interp_df['nbrcs_log'] - biases)/10))

In [ ]:
# CYGNSS - CURRENT AND WIND ESTIMATIONS
cygnss_and_interp_df['diff_nbrcs'] = cygnss_and_interp_df['nbrcs_log'] - predicted_nbrcs
cygnss_and_interp_df['diff_mss'] = (mss_from_cygnss - mss_from_wind_current)/mss_from_wind_current
cygnss_and_interp_df

In [ ]:
mic_df = retrieve_microplastics_gt().dropna()
mic_df

In [ ]:
interp_micro_mass = LinearNDInterpolator(list(zip(mic_df['sp_lon'], mic_df['sp_lat'])), mic_df['vansebillemodel_mass_log'])
interp_micro_abu = LinearNDInterpolator(list(zip(mic_df['sp_lon'], mic_df['sp_lat'])), mic_df['vansebillemodel_abundance_log'])


mass = interp_micro_mass(cygnss_and_interp_df['sp_lon'], cygnss_and_interp_df['sp_lat'])
cygnss_and_interp_df['van_sebille_mass'] = mass
cygnss_and_interp_df
df = cygnss_and_interp_df.iloc[: , 11:]
df2 = cygnss_and_interp_df.iloc[: , :2]
df['sp_lat'] = df2['sp_lat']
df['sp_lon'] = df2['sp_lon']
df_mss_ano_micro = df

df_mss_ano_micro

In [ ]:
plot_var(mic_df, 'vansebillemodel_mass_log')
plot_var(mic_df, 'vansebillemodel_abundance_log')

In [ ]:
if True:
    plot_var(cygnss_and_interp_df,'diff_nbrcs' )
    plot_var(cygnss_and_interp_df,'diff_mss' )
    plot_df = pd.DataFrame({'NBRCS_LOG_SCALED' : cygnss_and_interp_df['nbrcs_log'], 'Wind_Speed' : np.sqrt(cygnss_and_interp_df['u10']**2+cygnss_and_interp_df['v10']**2)})
    plot_df = plot_df[plot_df.NBRCS_LOG_SCALED > 10]
    plot_df = plot_df[plot_df.NBRCS_LOG_SCALED < 23]
    plot_df = plot_df[plot_df.Wind_Speed > 0.5]

    fig = px.density_heatmap(plot_df, y="NBRCS_LOG_SCALED", x="Wind_Speed", nbinsy=100 , color_continuous_scale=px.colors.sequential.Blackbody  ,title="2D Histogram plot of Wind speed and NBRCS in DB scale")

    fig.show()

    plt.style.use('seaborn-whitegrid')
    plt.plot((plot_df['Wind_Speed'].to_numpy()), (plot_df['NBRCS_LOG_SCALED'].to_numpy()), 'o', color='black');
    
    plt.show()

    era_5_df_one_hour = era_5_df.loc[(era_5_df['hours_since_ref'] == 254122.0)]
    cygnss_and_wind_df_one_hour = cygnss_and_interp_df.loc[(cygnss_and_interp_df['hours_since_ref'] == 254122.0)]

    plot_wind(era_5_df_one_hour, 'u10', 'v10')
    plot_wind(cygnss_and_wind_df_one_hour,  'u10', 'v10',interpolated= True)
    
    oskar_df_one_hour = oskar_df.loc[(oskar_df['hours_since_ref'] == 254112)]
    cygnss_and_wind_df_one_hour = cygnss_and_interp_df.loc[(cygnss_and_interp_df['hours_since_ref'] == 254122.0)]

    plot_wind(oskar_df_one_hour, 'u', 'v')
    plot_wind(cygnss_and_wind_df_one_hour,  'u', 'v', interpolated= True)

    plot_current_df = pd.DataFrame({'NBRCS_LOG_SCALED' : cygnss_and_interp_df['nbrcs_log'], 'Ocean_current_speed_15_meters_depth' : np.sqrt(cygnss_and_interp_df['u']**2+cygnss_and_interp_df['v']**2)})
    plot_current_df = plot_current_df[plot_current_df.NBRCS_LOG_SCALED > 10]
    plot_current_df = plot_current_df[plot_current_df.NBRCS_LOG_SCALED < 23]
    #plot_current_df = plot_current_df[plot_current_df.Ocean_current_speed_15_meters_depth < 1.1]
    fig = px.density_heatmap(plot_current_df, y="NBRCS_LOG_SCALED", x="Ocean_current_speed_15_meters_depth", title="2D Histogram plot of Ocean current strength at 15 meters depth and NBRCS in DB scale", color_continuous_scale=px.colors.sequential.Blackbody)
    fig.show()

    plt.style.use('seaborn-whitegrid')
    plt.plot((plot_current_df['Ocean_current_speed_15_meters_depth'].to_numpy()), (plot_current_df['NBRCS_LOG_SCALED'].to_numpy()), 'o', color='black');
    
    plt.show()
    cygnss_and_wind_df_one_hour['total_wind'] = np.sqrt(cygnss_and_wind_df_one_hour['u10']**2 + cygnss_and_wind_df_one_hour['v10']**2)

    fig = px.density_mapbox(cygnss_and_wind_df_one_hour, lat='sp_lat', lon='sp_lon', z='total_wind', radius=10,
                            center=dict(lat=0, lon=180), zoom=0,
                            mapbox_style="stamen-terrain")
    fig.show()
    plot_var(mic_df, 'vansebillemodel_mass_log')
    plot_var(mic_df, 'vansebillemodel_abundance_log')